In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import numpy as np
from common import *

In [12]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [13]:
from dataloaders import hip_dataset, knee_dataset
data = hip_dataset(args)

INFO - 2019-02-20 19:23:57,918 - Reading Structured data ...
INFO - 2019-02-20 19:23:58,132 - Reading Notes ...
INFO - 2019-02-20 19:23:59,402 - Stratifying ...


In [14]:
train_data = data.filter_data_length(data.get_data('train', structured=False), truncate=100)
dev_data = data.filter_data_length(data.get_data('dev', structured=False), truncate=100)

Pos Percentage [0.10457297]


INFO - 2019-02-20 19:24:00,125 - Maximum Sentence Length 610378.000000 , 100 percentile length 610378.000000 ... 
INFO - 2019-02-20 19:24:00,126 - Pos Percentage of remaining data ... 
INFO - 2019-02-20 19:24:00,127 - [0.10457297]
INFO - 2019-02-20 19:24:00,419 - Maximum Sentence Length 119250.000000 , 100 percentile length 119250.000000 ... 
INFO - 2019-02-20 19:24:00,419 - Pos Percentage of remaining data ... 
INFO - 2019-02-20 19:24:00,420 - [0.0875]


Pos Percentage [0.0875]


In [15]:
from PatientVec.models.baselines.LR import LR, LDA

lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : 'classifier', 'norm' : 'l2'})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)


Fit BOW Classifier ...
Fit TFIDF Classifier ...



BOW
{'accuracy': 0.8328125, 'roc_auc': 0.8195327788649707, 'pr_auc': 0.3475973485547539}


,0.0,1.0,micro avg,macro avg,weighted avg
f1-score,0.903,0.409,0.833,0.656,0.859
precision,0.963,0.296,0.833,0.630,0.905
recall,0.849,0.661,0.833,0.755,0.833
support,584.000,56.000,640.000,640.000,640.000


TFIDF
{'accuracy': 0.840625, 'roc_auc': 0.823752446183953, 'pr_auc': 0.37385566836057593}


,0.0,1.0,micro avg,macro avg,weighted avg
f1-score,0.909,0.378,0.841,0.643,0.862
precision,0.953,0.287,0.841,0.620,0.895
recall,0.868,0.554,0.841,0.711,0.841
support,584.000,56.000,640.000,640.000,640.000


In [7]:
lr.print_all_features(n=40)

loos onto attempt today partial infiltr stool thr intraop took surround smooth cortex great exposur extub pneumonia came diamet loosen fx swell heterotop communic lie fi prosthes erythromycin gram amount new tetracyclin underneath stat prosthet infect debrid suscept picc revis
----------
vertic septal synovial aforement great fen workup intraventricular haloperidol mom arthrogram cortex fibrous erythromycin fx tetracyclin cardiomegali disengag extub allograft lot came heterotop clamp gram platlet debrid communic subjac suscept prosthes lie underneath explant wndclt infect prosthet stat picc revis
----------


AttributeError: 'MultiOutputClassifier' object has no attribute 'estimators_'

Attention Experiments
======================

In [ ]:
from PatientVec.models.Vanilla import ClassificationTrainer as BasicCT
from PatientVec.models.Hierarchical import ClassificationTrainer as HierCT
from PatientVec.trainer import Trainer, Evaluator
from PatientVec.Experiments.modifiable_config_exp import vanilla_configs, attention_configs, hierarchical_configs, structured_configs

In [ ]:
from PatientVec.Experiments.hyperparam_exps import get_basic_data

In [ ]:
train_data, dev_data = get_basic_data(data, structured=False, truncate=98)

for e in attention_configs :
    config = e(data, structured=False, args=args)
    if args.output_dir is not None :
        config['exp_config']['basepath'] = args.output_dir
    config['training_config']['common']['bsize'] = 8
    config['training_config']['type'] = 'RMSprop'
    print(config)

    trainer = Trainer(BasicCT, config, _type=data.metrics_type, display_metrics=True)
    trainer.train(train_data, dev_data, n_iters=15, save_on_metric=data.save_on_metric)

    evaluator = Evaluator(BasicCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=True)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*300)

In [ ]:
train_data, dev_data = get_basic_data(data, structured=False, truncate=98)

for e in vanilla_configs :
    config = e(data, structured=False, args=args)
    if args.output_dir is not None :
        config['exp_config']['basepath'] = args.output_dir
    config['training_config']['common']['bsize'] = 8
    print(config)

    trainer = Trainer(BasicCT, config, _type=data.metrics_type, display_metrics=True)
    trainer.train(train_data, dev_data, n_iters=15, save_on_metric=data.save_on_metric)

    evaluator = Evaluator(BasicCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=True)
    _ = evaluator.evaluate(dev_data, save_results=True)
    print('='*300)

Saving Models
==============

In [ ]:
from common import generate_latex_tables
keys_to_use = ['1/precision', '1/recall', '1/f1-score', 'accuracy', 'roc_auc', 'pr_auc']
generate_latex_tables(data, keys_to_use)